In [1]:
#!pip install surprise
#!pip install import-ipynb
import os
import csv
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
from random import *
from surprise import accuracy
from collections import defaultdict
from surprise import KNNBasic
from collections import defaultdict
from operator import itemgetter
import NetflixLoadData as NetflixLoadData 
#sns.set_style("darkgrid")

In [2]:
use_pickle_file = True
max_n = 2500000 #how many rows we want from data_ratings and data_rating_plus_movie_title
reader = Reader(line_format='user item rating', rating_scale=(1, 5))

## Get data

In [3]:
NetflixLoadData.commented_implementation()

This part was a huge problem for me since the original files where weirdly structured
For example the ratings file was structured with the movie_id randomly being placed in a line and each line after that was refering to that movie without having the id in their line, then another movie randomly started in a line and lines after that were a reference to that movie

How it looked before:
1:
 1488844, 3, 2005-09-06 
 822109, 5, 2005-05-13 
 885013, 4, 2005-10-19 
 30878, 4, 2005-12-26 
....
....
...

2:
 2059652, 4, 2005-09-05 
 1666394, 3, 2005-04-19 
 1759415, 4, 2005-04-22 
 1959936, 5, 2005-11-21 
....
....
...

the way I fixed this was include everything in a dataframe and if there was a missing rating then that was a movie_id other wise it was a customer rating, then I combined all these rows to those ratingsm so each movie had a correct reference


(** to see all how these four files below were created, look at the file NetflixLoadData.py and there look at these functions -> get_dataframes(), create_dataframe() and commented_code_that_was_used_before() **)

In [4]:
data_movies, data_rating, data_rating_plus_movie_title, _ = NetflixLoadData.get_data_files(use_small_dataset=True)

In [5]:
data_rating

,movie_id,customer_id,rating
0,1,1488844,3.0
1,1,822109,5.0
2,1,885013,4.0
3,1,30878,4.0
4,1,823519,3.0
...,...,...,...
813413,4499,101554,4.0
813414,4499,1334851,3.0
813415,4499,1852040,1.0
813416,4499,185372,1.0


## Helper functions

In [6]:
def save_to_pickle(name, df):
    path_name = "pickle/"+name+".pickle"
    pickle_file = open(path_name,"wb")
    pickle.dump(df, pickle_file)
    pickle_file.close()

def load_pickle(name):
    path_name = "pickle/"+name+".pickle"
    return_input = open(path_name, "rb")
    return pickle.load(return_input)

## Finding information about the data before doing anything major

In [7]:
import MovieCustomerInformation as information
max_rating = 4
min_rating = 4

### Details about all customers/movies

In [ ]:
# get the average movie rating for all customers
# used to determine if this user typically gives bad or good reviews
# and then we can see if he really hates or loves a movie
all_customers_average_ratings =  information.all_average_ratings(df=data_rating, type='customer_id')
print(all_customers_average_ratings)

In [ ]:
all_movies_average_rating = information.all_average_ratings(df=data_rating, type='movie_id')
print(all_movies_average_rating)

In [ ]:
information.get_avg_rating_less_than(df=all_customers_average_ratings , max_rating=max_rating)

In [ ]:
information.get_avg_rating_higher_than(df=all_customers_average_ratings, min_rating=min_rating)

In [ ]:
information.get_avg_rating_less_than(df=all_movies_average_rating, max_rating=max_rating)

In [ ]:
information.get_avg_rating_higher_than(df=all_movies_average_rating, min_rating=min_rating)

In [ ]:
count_movies_rated_by_customer = information.all_get_rated_count(df=data_rating, type="customer_id")
count_movies_rated_by_customer

find customers who rated a random movie

In [ ]:
movie_to_find = data_movies['movie_title'][randrange(1000)]
information.get_customers_who_rated_movie_title(movie_title=movie_to_find)

In [ ]:
information.get_avg_rating_for_movie_title(movie_title=movie_to_find)

finding customers who rated a specific movie

In [ ]:

information.get_customers_who_rated_movie_title(movie_title="Harold and Kumar Go to White Castle")

In [ ]:
information.get_avg_rating_for_movie_title(movie_title="Harold and Kumar Go to White Castle")

In [ ]:
def generate_random_user_ids(from_n, to_n):
    ids = np.unique(data_rating['customer_id'])[from_n:to_n]
    print("random user id used while in development\n")
    return ids
generate_random_user_ids(1000, 1500)

### Details about a specific customer

In [10]:
# ids I like -> 28812, 56514, 56520, 1488844
customer_id_use = 534046 #84100

In [ ]:
#all_custumers_id_rows(customer_id=customer_id_use)
## same as line above but including movie_year and movie_title

#all_customers_id_plus_movie_title_rows(customer_id=customer_id_use)
information.all_id_rows(df=data_rating_plus_movie_title, type="customer_id", item_id=customer_id_use)

In [ ]:
#get_users_avg_rating(customer_id=customer_id_use)
information.get_item_avg_rating(df=all_customers_average_ratings, type='customer_id', item_id=customer_id_use)

In [ ]:
#display_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=4)
information.display_movies_customer_rated_higher_than(df=data_rating_plus_movie_title, customer_id=customer_id_use, min_rating=min_rating)

In [ ]:
information.display_movies_customer_rated_lower_than(df= data_rating_plus_movie_title,customer_id=customer_id_use, max_rating=max_rating)

In [ ]:
information.get_users_loved_hated_movies(df=data_rating_plus_movie_title, customer_id=customer_id_use, minmax_rating=4)

In [ ]:
all_customers_avg_rating = all_customers_average_ratings["avg_rating"].mean()

usr = information.all_average_ratings(df=data_rating, type='customer_id')
usr = usr[usr["customer_id"] == 596533]

change = ((usr["avg_rating"] - all_customers_avg_rating) / all_customers_avg_rating)*100


print("average rating for user 596533:", float(usr["avg_rating"]))
print("average rating for all users:", all_customers_avg_rating)
print("difference between them", round(float(change), 2), "%")

## Recommendations with collaborative filtering

#### functions

In [11]:
from surprise import dataset, KNNBaseline, accuracy
from surprise.model_selection import train_test_split, LeaveOneOut

def get_drop_list(type="movie_id"):
    #>IF MOVIE:  movie_id, movie rating count, movie rating mean
    #>IF CUSTOMER: customer_id, custumer rating count, customer rating mean
    df_count_mean_summary = data_rating.groupby(type)['rating'].agg(['count', 'mean'])
    df_count_mean_summary.index = df_count_mean_summary.index.map(int)
    #>IF MOVIE: returns 1799.0 as a benchmark number
    #IF CUSTOMER: returns 52.0 as a benchmark number
    benchmark = round(df_count_mean_summary['count'].quantile(0.7),0)
    # drop all rows below benchmark
    df_drop_list = df_count_mean_summary[df_count_mean_summary['count'] < benchmark]
    # return all indexes to drop
    return df_drop_list

def get_customer_recommendations(customer_id, predictor):
    #> returns movie_id, movie_year, movie_title
    chosen_customer_pred = data_movies.copy()
    # fails if movie_id is the index so we have to reset the index back to normal (0-N)
    chosen_customer_pred = chosen_customer_pred.reset_index()
    # makes sure that we only pick movies that are not in the movie dropped list
    chosen_customer_pred = chosen_customer_pred[~chosen_customer_pred['movie_id'].isin(df_movie_drop_list)]
    # make prediction for customer with id = <customer_id> and put it into 'estimated_score'
    chosen_customer_pred['estimated_score'] = chosen_customer_pred['movie_id'].apply(lambda x: predictor.predict(customer_id, x).est)
    # sort by 'estimated score'
    chosen_customer_pred = chosen_customer_pred.sort_values('estimated_score', ascending=False).set_index('movie_id')
    return chosen_customer_pred

# print out movies/tv show user has previously rated
def display_rated_content(customer_id=customer_id_use, number_to_show=20):
    print("Movies/TV Shows rated by customer", customer_id)
    #df = all_customers_id_plus_movie_title_rows(customer_id=customer_id)
    df = information.all_id_rows(df=data_rating_plus_movie_title, type='customer_id', item_id=customer_id)
    df = df[['movie_title', 'rating']].sort_values('rating', ascending=False)
    print(df[0:number_to_show].set_index('movie_title'))

def display_customers_recommendations(customer_id=customer_id_use, df=[], number_to_show=20):
    print("Movies/TV Shows recommended to customer")
    tmp_df = df[['movie_title', 'estimated_score']][0:number_to_show]
    tmp_df = tmp_df.set_index('movie_title')
    print(tmp_df)

def display_recommendation(customer_id, number_to_show, predictor):
    chosen_customer_pred = get_customer_recommendations(customer_id=customer_id, predictor=predictor)
    display_customers_recommendations(customer_id=customer_id, df=chosen_customer_pred, number_to_show=number_to_show)

def print_evaluation_accuracy(predictions):
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(predictions, verbose=False))
    print("MSE: ", accuracy.mse(predictions, verbose=False))
    print("MAE: ", accuracy.mae(predictions, verbose=False))
    # FCP = Fraction of Concordant Pairs
    print("FCP: ", accuracy.mae(predictions, verbose=False))

#### create recommenders

In [ ]:
## creating a list of the movies that we don't wanna include
df_movie_drop_list = get_drop_list(type="movie_id")
#df_customer_drop_list = get_summary(type="customer_id")

In [ ]:
def test_against_our_custom_function():
    _, data_rating, _, _ = NetflixLoadData.get_data_files(use_small_dataset=True)
    dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
    fullTrainset = dataset.build_full_trainset()

    trainSet, testSet = train_test_split(dataset, test_size=.25, random_state=1)

    algo = SVD(random_state=10)
    algo.fit(trainSet)

    predictions = algo.test(testSet)
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(predictions, verbose=False))
    print("MSE: ", accuracy.mse(predictions, verbose=False))
    print("MAE: ", accuracy.mae(predictions, verbose=False))
    # FCP = Fraction of Concordant Pairs
    print("FCP: ", accuracy.mae(predictions, verbose=False))


class convert_to_raw_ratings(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['customer_id'], df['movie_id'], df['rating'])]
        self.reader=reader

raw_ratings = convert_to_raw_ratings(data_rating, reader)
print("\nBuilding recommendation model...")
trainSet, testSet = train_test_split(raw_ratings, test_size=.25, random_state=1)

algo = SVD(random_state=10)
algo.fit(trainSet)
algo_predictions = algo.test(testSet)

In [ ]:
data_movies

In [ ]:
save_to_pickle("algorithm_svd", algo)
# #save_to_pickle("algo_raw_rating", raw_ratings)
# save_to_pickle("algo_trainSet", trainSet)
# save_to_pickle("algo_testSet", testSet)

In [ ]:
#cross_validate(algo, raw_ratings, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
svd = SVD(random_state=10)
data_new = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = data_new.build_full_trainset()
svd.fit(trainset)
svd_predictions = svd.test(testSet)
print_evaluation_accuracy(svd_predictions)


In [ ]:
#cross_validate(svd, data_new, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#surprise.get_neighbours(1, 10)

In [ ]:
##TODO: look into knows_user 
#       -> which tells if the user is part of the trainset,
#           -> equals true if user has at least one rating

### display recommendations

In [ ]:
number_to_show = 10

#### customer = 79724

In [ ]:
display_rated_content(customer_id=79724, number_to_show=number_to_show)

In [ ]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=svd)

In [ ]:
display_recommendation(customer_id=79724, number_to_show=number_to_show, predictor=algo)

#### customer = customer_id_use

In [ ]:
display_rated_content(customer_id=customer_id_use, number_to_show=number_to_show)

In [ ]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=svd)

In [ ]:
display_recommendation(customer_id=customer_id_use, number_to_show=number_to_show, predictor=algo)
# recommends for example: Reservoir Dogs, which the user has already rated

### print accuracy

In [ ]:
print_evaluation_accuracy(svd_predictions)

In [ ]:
print_evaluation_accuracy(algo_predictions)

In [ ]:
test_against_our_custom_function() #for algo predictions

results are the same so we will not use the custom function since it adds no value

## other

In [ ]:
def getMovieName(movieID):
      if int(movieID) in tmp_data_movies:
    return tmp_data_movies[int(movieID)]
  else:
      return ""

In [12]:
dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
fullTrainset = dataset.build_full_trainset()
fullTestSet = fullTrainset.build_anti_testset()
trainSet, testSet = train_test_split(dataset, test_size=.25, random_state=1)

In [13]:
# creating a dict of movie id and movie_title to make sure we don't recommend user something he has rated before
tmp_data_movies = data_movies[['movie_id', 'movie_title']]
tmp_data_movies = tmp_data_movies.set_index('movie_id').T
tmp_data_movies = tmp_data_movies.to_dict('list')
tmp_data_movies = {k: str(v[0]) for k,v in tmp_data_movies.items()}

In [14]:
data_movies

,movie_id,movie_year,movie_title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [15]:
tmp_data_movies

{1: 'Dinosaur Planet',
 2: 'Isle of Man TT 2004 Review',
 3: 'Character',
 4: "Paula Abdul's Get Up & Dance",
 5: 'The Rise and Fall of ECW',
 6: 'Sick',
 7: '8 Man',
 8: 'What the #$*! Do We Know!?',
 9: "Class of Nuke 'Em High 2",
 10: 'Fighter',
 11: 'Full Frame: Documentary Shorts',
 12: 'My Favorite Brunette',
 13: 'Lord of the Rings: The Return of the King: Extended Edition: Bonus Material',
 14: 'Nature: Antarctica',
 15: 'Neil Diamond: Greatest Hits Live',
 16: 'Screamers',
 17: '7 Seconds',
 18: 'Immortal Beloved',
 19: "By Dawn's Early Light",
 20: 'Seeta Aur Geeta',
 21: 'Strange Relations',
 22: 'Chump Change',
 23: "Clifford: Clifford Saves the Day! / Clifford's Fluffiest Friend Cleo",
 24: 'My Bloody Valentine',
 25: 'Inspector Morse 31: Death Is Now My Neighbour',
 26: 'Never Die Alone',
 27: "Sesame Street: Elmo's World: The Street We Live On",
 28: 'Lilo and Stitch',
 29: 'Boycott',
 30: "Something's Gotta Give",
 31: 'Classic Albums: Meat Loaf: Bat Out of Hell',
 32: 

In [16]:
similarity_matrix = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}).fit(fullTrainset).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [17]:
# generate_random_user_ids(1000, 1500)

In [18]:
# for i in generate_random_user_ids(1000, 1500):
#     if (len(all_custumers_id_rows(i)) > 3):
#         print(i)

In [19]:
test_subject = 596533#16272
k = 10

In [20]:
test_subject_iid = trainSet.to_inner_uid(test_subject)

test_subject_ratings = trainSet.ur[test_subject_iid]
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [21]:
def get_candidates(k_neighbours):
    candidates = defaultdict(float)
    for itemID, rating in k_neighbours:
        try:
            similarities = similarity_matrix[itemID]
            for innerID, score in enumerate(similarities):
                candidates[innerID] += score * (rating / 5.0)
        except:
            continue
    return candidates

candidates = get_candidates(k_neighbours)

In [24]:
def get_watched(trainset):
    watched = {}
    for itemID, rating in trainset.ur[test_subject_iid]:
        watched[itemID] = 1
    return watched
watched = get_watched(trainSet)


In [26]:
def get_recommendations(candidates):
    recommendations = []
    position = 0
    for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            recommendations.append(getMovieName(trainSet.to_raw_iid(itemID)))
            position += 1
            # only want top n which in our case in 10
            if(position > 10): break
    return recommendations
recommendations = get_recommendations(candidates)

In [27]:
for rec in recommendations:
    print("Movie: ", rec)

Movie:  200 Cigarettes
Movie:  A Program About Unusual Buildings & Other Roadside Stuff
Movie:  Nemesis
Movie:  Nature: Antarctica
Movie:  Warren Miller: Bloopers
Movie:  The Hunchback of Notre Dame
Movie:  Washington Heights
Movie:  My Man Godfrey
Movie:  Love Actually
Movie:  From Hell
Movie:  Midnight Panther


In [30]:
print("user", test_subject, 'previous recommendations')
display_recommendation(customer_id=test_subject, number_to_show=10, predictor=svd)

user 596533 previous recommendations


NameError: name 'svd' is not defined

In [ ]:
#TODO:
#for the new user problem
## find highest rated movie / tv shows that have been watched the most -> how likely is new user to click on it ?

In [ ]:
#cross_validate(similarity_matrix, dataset, measures=['MSE', 'RMSE', 'MAE'], cv=5, verbose=True)